In [1]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\t14\Downloads\archive (3)\station_hour.csv"
df = pd.read_csv(file_path)

# Preview
print("Shape:", df.shape)
df.info()
df.head()


C:\Users\t14\AppData\Local\Temp\ipykernel_18596\2005992110.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Shape: (2589083, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2589083 entries, 0 to 2589082
Data columns (total 16 columns):
 #   Column      Dtype  
---  ------      -----  
 0   StationId   object 
 1   Datetime    object 
 2   PM2.5       float64
 3   PM10        float64
 4   NO          float64
 5   NO2         float64
 6   NOx         float64
 7   NH3         float64
 8   CO          float64
 9   SO2         float64
 10  O3          float64
 11  Benzene     float64
 12  Toluene     float64
 13  Xylene      float64
 14  AQI         float64
 15  AQI_Bucket  object 
dtypes: float64(13), object(3)
memory usage: 316.1+ MB


,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,AP001,2017-11-24 17:00:00,60.50,98.00,2.35,30.80,18.25,8.50,0.1,11.85,126.40,0.1,6.10,0.10,NaN,NaN
1,AP001,2017-11-24 18:00:00,65.50,111.25,2.70,24.20,15.07,9.77,0.1,13.17,117.12,0.1,6.25,0.15,NaN,NaN
2,AP001,2017-11-24 19:00:00,80.00,132.00,2.10,25.18,15.15,12.02,0.1,12.08,98.98,0.2,5.98,0.18,NaN,NaN
3,AP001,2017-11-24 20:00:00,81.50,133.25,1.95,16.25,10.23,11.58,0.1,10.47,112.20,0.2,6.72,0.10,NaN,NaN
4,AP001,2017-11-24 21:00:00,75.25,116.00,1.43,17.48,10.43,12.03,0.1,9.12,106.35,0.2,5.75,0.08,NaN,NaN


In [2]:
df = df.dropna(subset=['AQI'])


In [3]:
df = df.drop(columns=['station', 'date', 'AQI_Bucket'], errors='ignore')


In [4]:
numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify categorical columns (columns with string values like 'PB001')
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create preprocessing steps for categorical and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Create a pipeline with preprocessing and model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Manual Model MAE:", mean_absolute_error(y_test, y_pred))
print("Manual Model R² Score:", r2_score(y_test, y_pred))

Manual Model MAE: 47.10479283288975
Manual Model R² Score: 0.7121979561133203


In [8]:
from pycaret.regression import *

# Re-import clean version of the dataset
df_ai = pd.read_csv(file_path)
df_ai = df_ai.dropna(subset=['AQI'])
df_ai = df_ai.drop(columns=['station', 'date', 'AQI_Bucket'], errors='ignore')

# Setup PyCaret
reg = setup(data=df_ai, target='AQI', session_id=123, normalize=True, html=False)

# AutoML: Compare all models
best_model = compare_models()


                    Description             Value
0                    Session id               123
1                        Target               AQI
2                   Target type        Regression
3           Original data shape     (2018893, 15)
4        Transformed data shape     (2018893, 15)
5   Transformed train set shape     (1413225, 15)
6    Transformed test set shape      (605668, 15)
7              Numeric features                12
8          Categorical features                 2
9      Rows with missing values             89.9%
10                   Preprocess              True
11              Imputation type            simple
12           Numeric imputation              mean
13       Categorical imputation              mode
14     Maximum one-hot encoding                25
15              Encoding method              None
16                    Normalize              True
17             Normalize method            zscore
18               Fold Generator             KFold


                                    Model       MAE         MSE      RMSE  \
knn                 K Neighbors Regressor   30.7335   3789.7288   61.5577   
lightgbm  Light Gradient Boosting Machine   36.9153   4252.7337   65.2100   
gbr           Gradient Boosting Regressor   39.5986   4790.6189   69.2103   
lr                      Linear Regression   50.3291   6060.2651   77.8442   
ridge                    Ridge Regression   50.3291   6060.2651   77.8442   
lar                Least Angle Regression   50.3291   6060.2651   77.8442   
br                         Bayesian Ridge   50.3291   6060.2648   77.8442   
llar         Lasso Least Angle Regression   50.3534   6078.0029   77.9579   
lasso                    Lasso Regression   50.3537   6078.0113   77.9579   
huber                     Huber Regressor   48.9078   6263.1328   79.1361   
dt                Decision Tree Regressor   40.2980   6444.6103   80.2736   
en                            Elastic Net   53.2284   6714.9158   81.9409   